Importing Libraries

In [23]:
import torch
from torch import nn, save, load
from torch.optim import Adam
import torchvision
from torchvision import transforms

Data Preprocesssing and Loading

In [24]:
#this is for data preprocessing and loading with train data
def train_pl():
  #the transofrmation we will apply to the images from the FER dataset
  transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor(), #convert image to tensor
    transforms.Normalize(.485, .229) # Normalize image
  ])

  # loading and downloading the dataset
  train_data = torchvision.datasets.FER2013(root='/content',split = 'train', transform=transform)

  #create dataloaders so that the FER data can be loaded into the model we will implement
  #batches of 19, shuffled, 2 extra worker processes (main + worker + worker = 3 worker streams)
  train_loader = torch.utils.data.DataLoader(train_data, batch_size=19, shuffle=True, num_workers=2)

  return train_loader


Create the Model



In [25]:
class EmotionModel(nn.Module):
  def __init__(self):
    super(EmotionModel, self).__init__()
    self.conv_layers = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(32, 64, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(64, 128, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
    )
    self.fc_layers = nn.Sequential(
        nn.Linear(128 * 6 * 6, 128),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(128, 7)  #  7 classes for different emotions
    )

  def forward(self, x):
    x = self.conv_layers(x)
    x = x.view(x.size(0), -1)
    x = self.fc_layers(x)
    return x

Training the model

In [26]:
def train():
  #creates a new instance of our model and sets it up on a GPU
  model = EmotionModel().to('cuda')

  #sets up an omptimizer called Adam
  optimizer = Adam(model.parameters(), lr = 1e-3)

  #Sets up a cross entropy loss function
  loss_fn = nn.CrossEntropyLoss()

  #sets up the training dataset
  train_set = train_pl()

  #training the model over and over again
  for epoch in range(50): #train for 50 epochs
    for batch in train_set:
      #input data, expecccted data
      x, y = batch

      #move data to GPU
      x, y = x.to('cuda'), y.to('cuda')

      #get our models prediction
      prediction = model(x)

      #get the loss
      loss = loss_fn(prediction, y)

      #reset the optimizer
      optimizer.zero_grad()

      #goes through every neuron in the model and change the weight to attempt to lower the loss
      loss.backward()

      #calculates new gradients
      optimizer.step()

    print(f"Epoch {epoch+1}\n---------------------------------")
    print(f"\tloss:{loss}")
    print("---------------------------------")


  # saving our model to our environment

  return model


In [22]:
trained_model = train()
save(trained_model, 'model_MK1')

Epoch 1
---------------------------------
	loss:1.1392178535461426
---------------------------------
Epoch 2
---------------------------------
	loss:1.2988755702972412
---------------------------------
Epoch 3
---------------------------------
	loss:1.0090185403823853
---------------------------------
Epoch 4
---------------------------------
	loss:0.9176467061042786
---------------------------------
Epoch 5
---------------------------------
	loss:0.9744395017623901
---------------------------------
Epoch 6
---------------------------------
	loss:1.426256537437439
---------------------------------
Epoch 7
---------------------------------
	loss:1.2133121490478516
---------------------------------
Epoch 8
---------------------------------
	loss:0.7597891688346863
---------------------------------
Epoch 9
---------------------------------
	loss:0.667921245098114
---------------------------------
Epoch 10
---------------------------------
	loss:1.55316960811615
---------------------------